In [1]:
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd
import numpy as np

Read the raw data and print few rows

In [22]:
df=pd.read_csv('/content/Indian Earthquakes List Update_Magnitudes (1).csv',skiprows=9,encoding = "ISO-8859-1")
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,6,7,9,10,11,12,13,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME,Unnamed: 5,MAGNITUDE,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,LAT (N),LONG (E),DEPTH (km),INTENSITY,Unnamed: 16,Unnamed: 17,LOCATION,REFERENCE
0,NaN,NaN,NaN,NaN,(UTC),(IST),Mw,Mw,Mb,Mb,Ms,ML,NaN,NaN,NaN,MM,MMI,MME,NaN,NaN
1,1.0,-2474.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
2,2.0,-325.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
3,3.0,25.0,0,0.0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,72.9,33.72,0.0,NaN,NaN,NaN,NaN,Dr STGR
4,4.0,26.0,5,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,5.698851894,5.999685205,17.3,80.1,NaN,NaN,NaN,NaN,NaN,NEIC


Warning received -  Columns (2,6,7,9,10,11,12,13,15,16,17) have mixed types

Print the number of rows

In [23]:
df.shape

(52990, 20)

Print the columns of the raw data

In [24]:
df.columns

Index(['Sl. No.', 'YEAR ', 'MONTH ', 'DATE', 'ORIGIN TIME', 'Unnamed: 5',
       'MAGNITUDE', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'LAT (N)', 'LONG (E)', 'DEPTH (km)', 'INTENSITY',
       'Unnamed: 16', 'Unnamed: 17', 'LOCATION', 'REFERENCE'],
      dtype='object')

Pick required columns only. Apply filter based on columns required to create a new data

In [25]:
df_filter=df[['YEAR ', 'MONTH ', 'DATE', 'ORIGIN TIME', 'MAGNITUDE', 'LAT (N)', 'LONG (E)', 'DEPTH (km)']]
#df_filter = df_filter.iloc[1:] #Skip the first row with Column Description
df_filter.drop(0)
df_filter=df_filter[df_filter['YEAR ']>1000] #Filter for year grater than 1000
df_filter.head()

,YEAR,MONTH,DATE,ORIGIN TIME,MAGNITUDE,LAT (N),LONG (E),DEPTH (km)
15,1053.0,00,0.0,NaN,7.0,69.13,32.85,0.0
16,1063.0,12,5.0,NaN,NaN,19.1,73.2,NaN
17,1100.0,00,0.0,NaN,8.5,85,27.5,0.0
18,1102.0,02,28.0,NaN,5.6,62.2,34.4,0.0
19,1294.0,00,0.0,NaN,7.5,77.2,30.5,0.0


In [26]:
df_filter.shape

(52975, 8)

Rename the columns to avoid spaces

In [27]:
req_columns=['YEAR', 'MONTH', 'DAY', 'ORIGIN_TIME', 'MAGNITUDE','LAT', 'LONG', 'DEPTH_KM']

In [28]:
df_filter.columns=req_columns
df_filter.columns

Index(['YEAR', 'MONTH', 'DAY', 'ORIGIN_TIME', 'MAGNITUDE', 'LAT', 'LONG',
       'DEPTH_KM'],
      dtype='object')

In [29]:
df_filter.shape

(52975, 8)

In [30]:
df_filter.to_csv('earthquakes1.csv', encoding='utf-8', index=False)

In [31]:
missing_values_count = df_filter.isnull().sum()
missing_values_count

YEAR               0
MONTH             18
DAY               57
ORIGIN_TIME    31791
MAGNITUDE      12054
LAT                0
LONG               0
DEPTH_KM        2177
dtype: int64

Based on number of Nulls in ORIGIN_TIME , more than 50% of samples have nulls. Therfore dropping this column as it cannot contribute to the data analysis

In [32]:
df_filter.drop('ORIGIN_TIME', 1,inplace=True)

In [33]:
df_filter.shape
df_filter.dropna(subset=['DAY','MONTH'], how='all',inplace=True)
df_filter = df_filter[df_filter['DAY'].notna() ]
df_filter = df_filter[df_filter['MONTH'].notna() ]
df_filter = df_filter[(df_filter[['DAY','MONTH']] != 0).all(axis=1)]

df_filter=df_filter[~df_filter['MONTH'].isnull()]
df_filter=df_filter[~df_filter['DAY'].isnull()]

In [34]:
df_filter.DAY.isnull().sum(),df_filter.DAY.isna().sum()

(0, 0)

In [ ]:
df_filter.shape

(52889, 7)

Replace Depth with mode where ever missing

In [35]:
df_filter['DEPTH_KM'].fillna(df_filter['DEPTH_KM'].mode()[0], inplace=True)

In [36]:
missing_values_count = df_filter.isnull().sum()
missing_values_count

YEAR             0
MONTH            0
DAY              0
MAGNITUDE    12008
LAT              0
LONG             0
DEPTH_KM         0
dtype: int64

Replace missing Magnitude with Average

In [37]:
df_filter['MAGNITUDE']=df_filter['MAGNITUDE'].astype(float)
df_filter['MAGNITUDE'].fillna(df_filter['MAGNITUDE'].mean(), inplace=True)

Drop the inconsistent lat/long notations. Making an assumption that all lat longs are in N/E Hemisphere based on Geographic region under study

In [38]:
searchfor = ['°', 'E','W','N','S']
df_filter=  df_filter[~df_filter.LAT.str.contains('|'.join(searchfor),na=False)]
df_filter=  df_filter[~df_filter.LONG.str.contains('|'.join(searchfor),na=False)]

COnvert LAT/LONG from string to float 

In [39]:
df_filter['LAT']=df_filter['LAT'].astype(float)
df_filter['LONG']=df_filter['LONG'].astype(float)

Check if LAT/LONG are in ideal range (-90,90) and (-180,180) respectively

In [40]:
df_filter['LAT'].min(),df_filter['LAT'].max()

(-10.9, 100.0)

LAT column has some invalid data. Drop those rows

In [41]:
df_filter=df_filter[df_filter['LAT']>0 ] 
df_filter=df_filter[df_filter['LAT']<90] 

In [42]:
df_filter['LONG'].min(),df_filter['LONG'].max()

(0.8, 179.9)

In [43]:
df_filter=df_filter[~df_filter['MONTH'].isnull()]
df_filter = df_filter[df_filter['MONTH'].notna() ]

df_filter=df_filter.dropna(subset=['MONTH'])

df_filter = df_filter[pd.notnull(df_filter['MONTH'])]

df_filter[ pd.to_numeric(df_filter['MONTH'], errors='coerce').isnull()]

,YEAR,MONTH,DAY,MAGNITUDE,LAT,LONG,DEPTH_KM
3110,1964.0,,7.0,5.05853,27.5,93.0,0.0
3111,1964.0,,7.0,5.05853,30.1,98.9,33.0


In [ ]:
df_filter[df_filter['MONTH'] == ''].index
text_empty =df_filter['MONTH'].str.len() > -1
df_filter.loc[text_empty].index

Int64Index([   16,    18,    21,    25,    26,    27,    29,    32,    33,
               35,
            ...
            32754, 32755, 32756, 32757, 32759, 32760, 32761, 32762, 32764,
            32767],
           dtype='int64', length=26660)

In [44]:
df_filter['YEAR']=df_filter['YEAR'].astype(int)

In [45]:
df_filter['DAY']=df_filter['DAY'].astype(int)

In [46]:
df_filter=df_filter[~df_filter['MONTH'].isnull()]
df_filter['MONTH']=df_filter['MONTH'].astype(int)

ValueError: ignored

In [47]:
df_filter['DT']=pd.to_datetime(df_filter[['YEAR', 'MONTH', 'DAY']])

ValueError: ignored

In [ ]:
df_filter.to_csv('earthquakes2.csv', encoding="utf-8", index=False)
df_filter.shape

In [ ]:
missing_values_count = df_filter.isnull().sum()
missing_values_count

Notice that after cleaning the dataset, there are no null values . We can now use this data in the ML algorithms

In [ ]:
#No of rows removed,#No of columns removed
df.shape[0]-df_filter.shape[0],df.shape[1]-df_filter.shape[1]

Percentage of rows dropped after data cleaning

In [ ]:
#% of rows removed
((df.shape[0]-df_filter.shape[0])/df.shape[0])*100.0